In [85]:
import pandas as pd
import random

In [86]:
# World Cup Matches
matches = pd.read_csv("../WorldCupMatches.csv")

# World Cup Players
players = pd.read_csv("../WorldCupPlayers.csv")

# World Cups
cups = pd.read_csv("../WorldCups.csv")

In [87]:
matches.head()

,Year,Datetime,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,Home Team Initials,Away Team Initials
0,2010.0,11 Jun 2010 - 16:00,Group A,South Africa,1.0,1.0,Mexico,84490.0,0.0,0.0,Ravshan IRMATOV (UZB),ILYASOV Rafael (UZB),KOCHKAROV Bakhadyr (KGZ),RSA,MEX
1,2010.0,11 Jun 2010 - 20:30,Group A,Uruguay,0.0,0.0,France,64100.0,0.0,0.0,NISHIMURA Yuichi (JPN),SAGARA Toru (JPN),JEONG Hae Sang (KOR),URU,FRA
2,2010.0,12 Jun 2010 - 13:30,Group B,Korea Republic,2.0,0.0,Greece,31513.0,1.0,0.0,HESTER Michael (NZL),HINTZ Jan Hendrik (NZL),MAKASINI Tevita (TGA),KOR,GRE
3,2010.0,12 Jun 2010 - 16:00,Group B,Argentina,1.0,0.0,Nigeria,55686.0,1.0,0.0,Wolfgang STARK (GER),SALVER Jan-Hendrik (GER),PICKEL Mike (GER),ARG,NGA
4,2010.0,12 Jun 2010 - 20:30,Group C,England,1.0,1.0,USA,38646.0,1.0,1.0,SIMON Carlos (BRA),HAUSMANN Altemir (BRA),BRAATZ Roberto (BRA),ENG,USA


In [88]:
players.head()

,Team Initials,Coach Name,Player Name
0,USA,MILLAR Bob (USA),Tom FLORIE
1,BRA,DE CARVALHO Pindaro (BRA),PREGUINHO
2,PER,BRU Francisco (ESP),Placido GALINDO
3,PAR,DURAND LAGUNA Jose (ARG),Luis VARGAS PENA
4,CHI,ORTH Gyorgy (HUN),Guillermo SUBIABRE


# Add new columns for players

In [89]:
cols = matches.columns
cols = list(cols)

new = [
    "Home Coach Name",
    "Home Player 1", "Home Player 2", 
    "Home Player 3", "Home Player 4", 
    "Home Player 5", "Home Player 6", 
    "Home Player 7", "Home Player 8", 
    "Home Player 9","Home Player 10",
    "Home Player 11",

    "Away Coach Name",
    "Away Player 1", "Away Player 2", 
    "Away Player 3", "Away Player 4", 
    "Away Player 5", "Away Player 6", 
    "Away Player 7", "Away Player 8", 
    "Away Player 9","Away Player 10",
    "Away Player 11"
]

cols += new
final = pd.DataFrame(columns=cols)

In [90]:
for i in range(len(matches)):
    home = ["coach"]
    away = ["coach"]
    # ID = matches.iloc[i]["MatchID"]
    HomeInitials = matches.iloc[i]["Home Team Initials"]
    AwayInitials = matches.iloc[i]["Away Team Initials"]   

    for j in range(len(players)):
        # IDPlayers = players.iloc[j]["MatchID"]
        player = players.iloc[j]["Player Name"]
        coach = players.iloc[j]["Coach Name"]
            
        if HomeInitials == players.iloc[j]["Team Initials"]:
            home[0] = coach
            home.append(player)
        elif AwayInitials == players.iloc[j]["Team Initials"]:
            away[0] = coach
            away.append(player)
        else:
            pass
        
    while len(home) != 12:
        if len(home) > 12:
            del home[random.randint(1, len(home) - 1)]
        else:
            home.append(random.choice(home))
        
        
    while len(away) != 12:
        if len(away) > 12:
            del away[random.randint(1, len(away) - 1)]
        else:
            away.append(random.choice(away))
    
    data = list(matches.iloc[i])
    data += home
    data += away
    final = final.append(pd.Series(data, index=cols), ignore_index=True)

In [92]:
final.head()

,Year,Datetime,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Attendance,Half-time Home Goals,Half-time Away Goals,...,Away Player 2,Away Player 3,Away Player 4,Away Player 5,Away Player 6,Away Player 7,Away Player 8,Away Player 9,Away Player 10,Away Player 11
0,2010.0,11 Jun 2010 - 16:00,Group A,South Africa,1.0,1.0,Mexico,84490.0,0.0,0.0,...,Arturo VAZQUEZ,Jose REYES,Pedro SOTO,Tomas BOY,Alberto GARCIA ASPE,O. PEREZ,R. MARQUEZ,G TORRADO,C BLANCO,R MARQUEZ
1,2010.0,11 Jun 2010 - 20:30,Group A,Uruguay,0.0,0.0,France,64100.0,0.0,0.0,...,Michel PLATINI,Didier DESCHAMPS,ZIDANE,VIEIRA,EVRA,A. DIARRA,Jean Paul BERTRAND DEMANES,Didier DESCHAMPS,ZIDANE,ZIDANE
2,2010.0,12 Jun 2010 - 13:30,Group B,Korea Republic,2.0,0.0,Greece,31513.0,1.0,0.0,...,KARAGOUNIS,KATSOURANIS,KARNEZIS,GLYKOS,KATSOURANIS,KARNEZIS,KARNEZIS,Tassos MITROPOULOS,KARNEZIS,GLYKOS
3,2010.0,12 Jun 2010 - 16:00,Group B,Argentina,1.0,0.0,Nigeria,55686.0,1.0,0.0,...,UCHE,ENYEAMA,KANU,YOBO,KANU,ENYEAMA,UCHE,ENYEAMA,KESHI Stephen (NGA),ENYEAMA
4,2010.0,12 Jun 2010 - 20:30,Group C,England,1.0,1.0,USA,38646.0,1.0,1.0,...,Tony MEOLA,Michael WINDISCHMANN,Thomas DOOLEY,STEWART,FRIEDEL,REYNA,BOCANEGRA,DEMPSEY,Michael WINDISCHMANN,Michael WINDISCHMANN


# Save final into CSV file

In [93]:
final.to_csv("../final.csv")